In [1]:
from random import randint
from PIL import Image
import numpy as np
import pickle
import pycuda.driver as cuda
import pycuda.autoinit
import uff
import tensorrt as trt

from matplotlib import pyplot as plt
from six.moves import cPickle

In [2]:
## information about data needed to parse model and build engine
class ModelData(object):
    MODEL_FILE = "/home/vtpc/Documents/Alvils/tensorrt/pretrained-models/cifar-10-googlenet/standalone.pb"
    INPUT_NAME ="Input1"
    INPUT_SHAPE = (3, 32, 32) # always nchw
    OUTPUT_NAME = "prob"
    DTYPE = trt.float32

In [3]:
# tensorflow to uff
uff_model = uff.from_tensorflow_frozen_model(ModelData.MODEL_FILE, [ModelData.OUTPUT_NAME])
TRT_LOGGER = trt.Logger(trt.Logger.INFO)

=== Automatically deduced input nodes ===
[name: "Input1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: 1
      }
      dim {
        size: 32
      }
      dim {
        size: 32
      }
      dim {
        size: 3
      }
    }
  }
}
]

Using output node prob
Converting to UFF graph
No. nodes: 318


In [ ]:
# parses model to trt
with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.UffParser() as parser:
    # builder argums for optimization
    builder.max_batch_size = 450
    builder.max_workspace_size = 1 << 30    
    # Parse the Uff Network
    parser.register_input(ModelData.INPUT_NAME, ModelData.INPUT_SHAPE)#, trt.UffInputOrder.NHWC)
    parser.register_output(ModelData.OUTPUT_NAME)
    parsed = parser.parse_buffer(uff_model, network)
    
    
    model_tensors = parser.parse(deploy=deploy_file, model=model_file, network=network, dtype=ModelData.DTYPE)
    # For Caffe, we need to manually mark the output of the network.
    # Since we know the name of the output tensor, we can find it in model_tensors.
    network.mark_output(model_tensors.find(ModelData.OUTPUT_NAME))
    
    # builds engine
    with builder.build_cuda_engine(network) as engine:
        with open("fp32.engine", "wb") as f:
            f.write(engine.serialize())


In [ ]:
print("done")